In [1]:
#download and unzip data
'''
!wget -P ./data http://www.statmt.org/europarl/v7/europarl.tgz
!tar -xvzf ./data/europarl.tgz -C ./data/
!pip install pandas
'''

'\n!wget -P ./data http://www.statmt.org/europarl/v7/europarl.tgz\n!tar -xvzf ./data/europarl.tgz -C ./data/\n!pip install pandas\n'

In [53]:
import pandas as pd
import os
import re
import sys

In [3]:
cols = ['lang', 'path']
df = pd.DataFrame(columns=cols)

languages = os.popen('ls ./data/txt/').read().split('\n')[:-1]
for lang in languages:
    print('adding language {}...'.format(lang))
    lang_frame = pd.DataFrame(columns=cols)
    txts = os.popen('ls ./data/txt/{}/'.format(lang)).read().split('\n')[:-1]
    paths = ['./data/txt/{}/{}'.format(lang, txt) for txt in txts]
    lang_frame['path'] = paths
    lang_frame['lang'] = lang
    lang_frame.index = range(len(df), len(df)+len(lang_frame))
    df = pd.concat([df, lang_frame], axis=0)

adding language bg...
adding language cs...
adding language da...
adding language de...
adding language el...
adding language en...
adding language es...
adding language et...
adding language fi...
adding language fr...
adding language hu...
adding language it...
adding language lt...
adding language lv...
adding language nl...
adding language pl...
adding language pt...
adding language ro...
adding language sk...
adding language sl...
adding language sv...


In [44]:
def update_progress(progress):
    """Displays or updates a console progress bar
    Accepts a float between 0 and 1. Any int will be converted to a float.
    A value under 0 represents a 'halt'.
    A value at 1 or bigger represents 100%
    """
    barLength = 25 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rProgress: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), round(progress*100, 3), status)
    sys.stdout.write(text)
    sys.stdout.flush()

def get_text(idx, df=df):
    """Takes idx and df and returns text."""
    path = df.loc[idx].path
    with open(path, 'r') as f:
        result = f.read()
    return result

def remove_tags(input_string):
    result = input_string
    tag = re.compile(r'<[^<]*>')
    while tag.search(result):
        match = tag.search(result)
        strt = match.span()[0]
        stp = match.span()[1]
        result = result[:strt] + result[stp:]
    return result

In [ ]:
#withold some fraction from every class for validation.
test_prop = .05
test = pd.DataFrame(columns=cols)
for lang in df.lang.unique():
    samp = df[df.lang == lang].sample(frac=test_prop)
    test = pd.concat([test, samp])
    
train = df[~df.index.isin(test.index)]
print('documents in training set: ', len(train))
print('documents in validation set: ', len(test))

def concat_to_file(idx, df, path):
    txt = get_text(idx, df=df)
    txt = remove_tags(txt)
    with open(path, 'a+') as f:
        f.write(txt)

train_path = './data/train.txt'
val_path = './data/val.txt'
os.system('rm {} {}'.format(train_path, val_path))

cnt = 0
print('creating train file...')
shuffle = train.sample(frac=1)
for i, row in shuffle.iterrows():
    try:
        concat_to_file(i, train, train_path)
    except UnicodeDecodeError:
        pass
    cnt += 1
    prog = cnt/len(train)
    update_progress(prog)
print('train file saved at: ', train_path)

cnt = 0
print('creating validation file...')
shuffle = test.sample(frac=1)
for i, row in shuffle.iterrows():
    try:
        concat_to_file(i, test, val_path)
    except UnicodeDecodeError:
        pass
    cnt += 1
    prog = cnt/len(test)
    update_progress(prog)
    
print('validation file saved at: ', val_path)

documents in training set:  177718
documents in validation set:  9354
creating train file...
Progress: [################---------] 64.315% 